# Serial Queue Simulation

## Authors:
> Frederico Lacis

> Daniel Carlier

In [201]:
import numpy as np
from numpy import random
import scipy.stats
import math

from datetime import date

import csv
import os
import pandas as pd

In [185]:
today = date.today().strftime('%d/%m/%Y')

### Auxiliary Functions

In [186]:
def hours(t):
    s = int(t*3600)
    m = int(s/60)
    h = int(m/60)
    return int(h)

In [187]:
def minutes(t):
    s = int(t*3600)
    m = int(s/60)
    h = int(m/60)
    return int(m-60*h)

In [188]:
def seconds(t):
    s = int(t*3600.0)
    m = int(s/60)
    return int(s-60*m)

In [189]:
def by_entry(e):
    return e['entry']

### Exponential random variable with gamma rate

In [190]:
def exponential_distribution(gamma):
    x = np.random.sample(1)
    return - math.log(x)/gamma

### Rate(t)

In [191]:
def rate(t):
    if(t < 1.0):
        return 10.0
    elif (t < 2.0):
        return 5.0
    elif (t < 3.0):
        return 10.0
    elif (t < 4.0):
        return 2.0
    elif (t < 5.0):
        return 5.0
    else:
        return 10.0

### Non-Homogeneous Poisson Process with Rate(t)

In [192]:
def non_homogeneous_poisson(rate,ratemax,closing_time):
    global logs
    global log
    people_id = 0
    entry_times = []
    entry = 0.0
    while (1):
        Z = exponential_distribution(ratemax)
        entry += Z

        if (entry < closing_time):
            U = np.random.sample(1)
            
            if (U < rate(entry)/ratemax):
                people_id += 1
                entry_times.append(dict(entry=entry,person=people_id))
                global today
                hour = hours(entry)
                minute = minutes(entry)
                second = seconds(entry)
                logs.append(dict(entry=entry,message = "%s: %2s:%2s:%2s, %d, Entered the system" % (today,str(hour).zfill(2),str(minute).zfill(2),str(second).zfill(2),people_id)))
        else:
            break

    return (people_id,entry_times)

### Serial Queue
> Returns idle time and list of updated entry times.

In [193]:
def serial_queue(rate, ratemax, closing_time, entry_times, cashier_amounts, gamma, people_amount, stage):
    global logs
    global log
    entry_times.sort(key=by_entry)
    idle_time = 0.0
    available = np.zeros(cashier_amounts)

    for person in range(people_amount) :
        attendance_duration = exponential_distribution(gamma)
        cashier = np.argmin(available)
        idle_time += max(entry_times[person]['entry'] - available[cashier], 0.0)
        available[cashier] = max(entry_times[person]['entry'],available[cashier])
        global today
        hour = hours(available[cashier])
        minute = minutes(available[cashier])
        second = seconds(available[cashier])
        logs.append(dict(entry=available[cashier],message = "%s: %2s:%2s:%2s, %d, Answered by cashier %d at stage %d" % (today,str(hour).zfill(2),str(minute).zfill(2),str(second).zfill(2),person+1,cashier+1,stage+1)))
        available[cashier] += attendance_duration
        entry_times[person]['entry'] = available[cashier]
 
    return idle_time, entry_times

### Calculate Idle Time
> Returns an array with idle time at each stage.

In [194]:
def calculate_idle_time (rate, rate_max, closing_time, stages, cashier_amounts, gamma):
# T -> closing_time
# m -> cashier_amounts
# n ->  stages
    global logs
    global log
    global today
   
    people_amount, entry_times = non_homogeneous_poisson(rate,rate_max,closing_time)
    idle_by_stage = np.zeros(stages)
    idle_by_stage[0] += entry_times[0]['entry']

    for stage in range(stages) :
        idle_time, entry_times = serial_queue(rate, rate_max, closing_time, entry_times, cashier_amounts[stage], gamma, people_amount, stage)
        entry_times.sort(key=by_entry)
        idle_by_stage[stage] += idle_time

    for person in range(people_amount) :
        second = seconds(entry_times[person]['entry'])
        minute = minutes(entry_times[person]['entry'])
        hour = hours(entry_times[person]['entry'])
        logs.append(dict(entry=entry_times[person]['entry'],message = "%s: %2s:%2s:%2s, %d, Got out of the system" % (today,str(hour).zfill(2),str(minute).zfill(2),str(second).zfill(2),person+1)))        
    return idle_by_stage

> ## Running

In [229]:
global logs
logs = []

# Parameters
gamma = 1.5
rate_max = 10.0
closing_time = 6.0
stages = 7
cashier_amounts = [5,4,3,2,1,9,19] 

results = calculate_idle_time (rate, rate_max, closing_time, stages, cashier_amounts, gamma)

print('Idle time for each stage:\n')
for result in results:
    print('\t • ' + str(result))
    
logs.sort(key=by_entry)

Idle time for each stage:

	 • 6.6464607677929575
	 • 11.990088178119773
	 • 7.594078142945912
	 • 4.712184998339385
	 • 3.3589974383478243
	 • 234.55012322785683
	 • 473.82970563442575


> ## Saving csv file

In [235]:
directory = './csv-results'

if not os.path.exists(directory):
    os.makedirs(directory)

# s = stages
# ca = cashier_amouts
# ct = closing_time
# rm = rate_max
filename = 'queue_s' + str(stages) \
        + '_ca' + '-'.join(str(x) for x in cashier_amounts) \
        + '_ct' + str(closing_time) \
        + '_rm' + str(rate_max) \
        + '.csv'

with open(os.path.join(directory, filename), 'w') as csvfile:
    filewriter = csv.writer(csvfile, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    
    filewriter.writerow(['Timestamp','PersonID','Activity'])
    for log in logs :
        log_msg = log['message'].split(',')
        filewriter.writerow([log_msg[0].strip(), log_msg[1].strip(), log_msg[2].strip()])

> ## Reading cvs file

In [236]:
pd.read_csv(os.path.join(directory, filename))

,Timestamp,PersonID,Activity
0,06/12/2019: 00:14:32,1,Entered the system
1,06/12/2019: 00:14:32,1,Answered by cashier 1 at stage 1
2,06/12/2019: 00:32:39,2,Entered the system
3,06/12/2019: 00:32:39,2,Answered by cashier 2 at stage 1
4,06/12/2019: 00:39:07,3,Entered the system
...,...,...,...
346,06/12/2019: 30:16:34,38,Answered by cashier 1 at stage 7
347,06/12/2019: 30:38:38,39,Answered by cashier 3 at stage 6
348,06/12/2019: 31:48:51,39,Answered by cashier 19 at stage 7
349,06/12/2019: 33:02:17,38,Got out of the system


## Tolerance Function
> Returns the averages, variances, and numbers of interactions for each stage.

In [198]:
def rSQtol2(tol, alpha, rate, rate_max, fechamento, stages, cashier_amounts, gamma):
    global zab2
    zab2 = scipy.stats.norm.ppf(1.0-alpha/2.0)

    variances = np.zeros(stages)
    means = calculate_idle_time (rate, rate_max, fechamento, stages, cashier_amounts, gamma)
    
    for i in range(1,100):
        idle_by_stage = calculate_idle_time (rate, rate_max, fechamento, stages, cashier_amounts, gamma)
        for j in range(stages) :
            mean = means[j]
            means[j] = means[j] + (idle_by_stage[j]-means[j])/(i+1)
            variances[j] = (1.0-1.0/i)*variances[j] + (i+1.0)*(means[j]-mean)**2

    number_of_iterations = np.full(stages,3000)
    
    for i in range(stages):
        while( 2.0*(variances[i]/number_of_iterations[i])*zab2*zab2 > tol**2 ):
            idle_by_stage = calculate_idle_time (rate, rate_max, fechamento, stages, cashier_amounts, gamma)
            mean = means[j]
            means[i] = means[i] + (idle_by_stage[i]-means[i])/(number_of_iterations[i]+1)
            variances[i] = (1.0-1.0/number_of_iterations[i])*variances[i] + (number_of_iterations[i]+1.0)*(means[i]-mean)**2
            number_of_iterations[i] += 1

    return means,variances,number_of_iterations

> ## Running

In [199]:
tol = 1
alpha = 0.05
means, variances, number_of_iterations = rSQtol2(tol, alpha, rate, rate_max, closing_time, stages, cashier_amounts, gamma)

for stage in range(stages) :
    print("Queue to stage",stage+1,": the average is in range ["+str(means[stage]-math.sqrt(variances[stage]/number_of_iterations[stage])*zab2)+","+str(means[stage]+math.sqrt(variances[stage]/number_of_iterations[stage])*zab2)+"] with probability of "+ str(1.0-alpha)+".")

Queue to stage 1 : the average is in range [6.4616051433440465,6.713050806283286] with probability of 0.95.
Queue to stage 2 : the average is in range [11.578004946757328,11.849371877791551] with probability of 0.95.


## Results

De forma a melhor analisar visualmente o tempo ocioso dos caixas, foi utilizado o programa Disco para gerar uma simulação das pessoas nas filas nos diferentes estágios.

Além deste notebook, enviamos em anexo um arquivo compactado que contém vídeos de simulações feitas no Disco e os respectivos arquivos csv que as geraram.

A simulação em questão utilizada para esta análise conta com 10 estágios, sendo que o número de caixas é organizado da seguinte maneira: 
* Estágio 1: 1 caixa
* Estágio 2: 2 caixas
* Estágio 3: 3 caixas
* Estágio 4: 1 caixa
* Estágio 5: 2 caixas
* Estágio 6: 3 caixas
* Estágio 7: 1 caixa
* Estágio 8: 2 caixas
* Estágio 9: 3 caixas
* Estágio 10: 1 caixa

Através da simulação, é possível observar que, devido ao fato do primeiro estágio possuir apenas um caixa, há um afunilamento de pessoas, gerando um aumento significativo da fila e um aumento no tempo ocioso dos caixas do estágio seguinte. No entanto, nos dois próximos estágios, devido a possuirem mais caixas, observa-se que o tempo de espera é significativamente menor. Em seguida, no próximo estágio, há um afunilamento novamente devido a uma redução do número de caixas para 1, gerando um aumento no tempo das filas como de esperado e do tempo de ociosidade.

Ao analisar diferentes cenários com diferentes relações de caixas por estágio, é possível observar um ponto interessante. Em casos onde há mais estágios do que caixas, como é o caso do arquivo com a simulação Disco apresentada, observa-se um acúmulo maior de pessoas em filas. Em contrapartida, em simulações onde se prioriza o número de caixas por estágio, a situação se inverte, havendo filas menores. Dado um número fixo de caixas, 10, e ao dispô-los de maneiras diferentes, é possível observar as mudanças em relação aos tempos das filas:

##### 2 caixas distribuídos em 5 estágios
  * Tempo Ocioso Estágio 1: 0,41263799
  * Tempo Ocioso Estágio 2: 3,34144508
  * Tempo Ocioso Estágio 3: 3,83466648
  * Tempo Ocioso Estágio 4: 11,71537659
  * Tempo Ocioso Estágio 5: 11,91268424
  * Tempo Ocioso Total: 31,21681038

##### 5 caixas distribuídos em 2 estágios
  * Tempo Ocioso Estágio 1: 5,3404323
  * Tempo Ocioso Estágio 2: 7.6682722
  * Tempo Ocioso Total: 13,0087045

Como é possível observar através das simulações acima, em relação ao tempo de ociosidade dos caixas, é preferível dispô-los de forma a priorizar um maior número de caixas por estágio a um número maior de estágios por caixa.